In [ ]:
import pandas as pd

In [ ]:
заявка_от_клиента = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ДИПЛОМ/Заявка от клиента.xlsx')
предложение_от_поставщиков = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ДИПЛОМ/Предложение от поставщиков.xlsx')
доставка_другие_города = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ДИПЛОМ/Доставка из других городов.xlsx')

In [ ]:
заявка_от_клиента.head()

,№,Номенклатура,Количество
0,1,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3
1,2,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1
2,3,"КТР-90 ключ трубный, прямые губки",1
3,4,Ключ разводной КР-30 окс\n\n\n\n\n,2
4,5,Фонарь прожект./кемп.,3


In [ ]:
предложение_от_поставщиков.head()

,№,Номенклатура,Количество (заявка),Поставщик,Город,Цена поставщика,Количество (поставщик),"Сроки поставки, дни"
0,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Все инструменты,Спб,499.00,3.0,1.0
1,NaN,NaN,NaN,Сатурн,Спб,603.84,3.0,1.0
2,NaN,NaN,NaN,Петрович,Спб,471.00,3.0,1.0
3,NaN,NaN,NaN,Гарвин,Спб,3504.00,3.0,3.0
4,2.0,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1.0,Сатурн,Спб,1403.58,1.0,1.0


In [ ]:
предложение_от_поставщиков.fillna(method='ffill', inplace=True)

In [ ]:
предложение_от_поставщиков.head()

,№,Номенклатура,Количество (заявка),Поставщик,Город,Цена поставщика,Количество (поставщик),"Сроки поставки, дни"
0,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Все инструменты,Спб,499.00,3.0,1.0
1,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Сатурн,Спб,603.84,3.0,1.0
2,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Петрович,Спб,471.00,3.0,1.0
3,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Гарвин,Спб,3504.00,3.0,3.0
4,2.0,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1.0,Сатурн,Спб,1403.58,1.0,1.0


На входе получаем набор офферов с номенклатурой и количеством; предложения от поставщиков по номенклатуре; затраты на доставку (offers, order_quantity, transport_costs)//
Создаем список поставщиков внутри функции suppliers = {} //

для каждого оффера берем его строку и разделяем на переменные все факторы

 for _, offer in offers.iterrows():
        supplier = offer['Поставщик']
        supplier_quantity = offer['Количество (поставщик)']
        supplier_price = offer['Цена поставщика']
        supplier_city = offer['Город']
//

Рассчитываем штраф за доставку, если город совпадает с нашим городом

delivery_penalty = transport_costs.loc[transport_costs['Город'] == supplier_city, 'Доставка'].values
delivery_penalty = delivery_penalty[0] if len(delivery_penalty) > 0 else 0
//

Если поставщика еще нет в созданом списке поставщиков - он добавляется с его факторами, стоимость 0 колво 0 цена = цена 
if supplier not in suppliers:
            suppliers[supplier] = {'Стоимость': 0, 'Количество': 0, 'Цена': supplier_price}
//

расчитываем полную стоимость = цена на колво + штраф за доставку  cost_with_delivery = supplier_price * supplier_quantity + delivery_penalty //

Если количество у поставщика в списке в сумме с количеством у поставщика в переменной меньше или равно, чем необходимо заказчику:
    в списке стоимость добавляется рассчитанная доставка с учетом штрафа за доставку 
    в списке количество добавляется кол-во из переменной поставщика
Иначе:
    рассчитывается минимальный поставщик по стоимости из словаря
    Если разница в цене переменной и цене у минимального поставщика меньше 1000, тогда:
        добавляем стоимость товаров к мин поставщику
        добавляем количество из новой поставки
    иначе:
    добавляем в словарь этого поставщика и его стоимость как штраф за доставку и количество

После завершения цикла перебора по всем офферам - выбирается поставщик с минимальной стоимостью

## АААААААААААААААААААААААААААААААААААААААААААААА

In [ ]:
def optimal_supplier(offers, order_quantity, transport_costs):
    suppliers = {}
    for _, offer in offers.iterrows():
        supplier = offer['Поставщик']
        supplier_quantity = offer['Количество (поставщик)']
        supplier_price = offer['Цена поставщика']
        supplier_city = offer['Город']
        delivery_penalty = transport_costs.loc[transport_costs['Город'] == supplier_city, 'Доставка'].values
        delivery_penalty = delivery_penalty[0] if len(delivery_penalty) > 0 else 0
        if supplier not in suppliers:
            suppliers[supplier] = {'Стоимость': 0, 'Количество': 0, 'Цена': supplier_price}
        cost_with_delivery = supplier_price * supplier_quantity + delivery_penalty
        if suppliers[supplier]['Количество'] + supplier_quantity <= order_quantity:
            suppliers[supplier]['Стоимость'] += cost_with_delivery
            suppliers[supplier]['Количество'] += supplier_quantity
        else:
            min_supplier = min(suppliers, key=lambda x: suppliers[x]['Стоимость'])
            if supplier_price - suppliers[min_supplier]['Цена'] < 1000:
                suppliers[min_supplier]['Стоимость'] += cost_with_delivery
                suppliers[min_supplier]['Количество'] += supplier_quantity
            else:
                suppliers[supplier]['Стоимость'] += delivery_penalty
                suppliers[supplier]['Количество'] += supplier_quantity
    optimal_supplier = min(suppliers.items(), key=lambda x: x[1]['Стоимость'])
    return optimal_supplier[0]

def calculate_optimal_suppliers(client_request, supplier_offers, other_city_delivery):
    optimal_suppliers = []
    for _, item in client_request.iterrows():
        offers_for_item = supplier_offers[supplier_offers['Номенклатура'] == item['Номенклатура']]
        optimal_suppliers.append({
            '№': item['№'],
            'Номенклатура': item['Номенклатура'],
            'Оптимальный поставщик': optimal_supplier(offers_for_item, item['Количество'], other_city_delivery)
        })
    result = pd.DataFrame(optimal_suppliers)
    return result

# Пример использования функции
результат = calculate_optimal_suppliers(заявка_от_клиента, предложение_от_поставщиков, доставка_другие_города)
print(результат)



   №                                    Номенклатура Оптимальный поставщик
0  1       Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n              Петрович
1  2  Ключ трубный CrV 90 градусов 2,0" 12\n\n\n\n\n                Сатурн
2  3               КТР-90 ключ трубный, прямые губки       Все инструменты
3  4              Ключ разводной КР-30 окс\n\n\n\n\n                Сатурн
4  5                          Фонарь прожект./кемп.               Петрович
5  6                Ключ разводной тонкие губки 50мм       Все инструменты


In [ ]:
определяем товар который нам нужен
проверяем и заносим в список всех поставщиков, которые подходя по товару, их количество и стоимость

сравниваем по количеству, есть ли поставщик, который в одиночку удовлетворяет заказ, предварительно определяем его( по наим суммарной стоимости)
даже если есть, проверяем комбинации заказов от разных поставщиков

рассчитываем стоимость заказа по всем уникальным поставщикам, то есть если мы берем все товары у одного, 
также учитываем есть ли вообще все товары только у одного поставщика, если нет - выбрасываем его из сравнения
определяем и выводим наименьший по стоимости заказ у одного поставщика

Потом рассчитываем все возможные комбинации по товарам отдельно, то есть просто все варианты выполнения заявки среди всех поставщиков
по типу: ключ трубный у "петровича", ключ разводной у "сатурна", фонарь у "петровича", микрофон для ульяны от "всех инструментов"

смотрим по стоимости и выбираем среди всех вариантов и наилучшего варианта с одним поставщиков, с учетом стоимости доставки

дополнительно: все комбинации по заказу товаров среди всех поставщиков, по типу 5 ключей 1-го типа у петровича, 4 ключей 1-го типа у сатурна, два фонаря от инструментов + 4 фонаря от петровича и т.д